<a href="https://www.kaggle.com/code/sergioper/titanic-predictions-pipelines-pca-gridsearch?scriptVersionId=99926746" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

## The Titanic dataset

The Titanic dataset is an example of a binary classification problem. Because we want to assign a (discrete) class label based in a set of features, with two possible outcomes: Survided or Not?
 
Classification belongs to the category of supervised learning where the targets also provided with the input data. 

More technical:
Classification predictive modeling is the task of approximating a mapping function (f) from input variables (X) to discrete output variables (y), (y =0 or y=1)

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
import scipy.stats as stats


from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler,StandardScaler,PolynomialFeatures,LabelEncoder,OneHotEncoder

from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error, mean_squared_log_error, confusion_matrix
from sklearn.metrics import explained_variance_score, median_absolute_error, max_error, classification_report, auc
from sklearn.metrics import roc_auc_score, recall_score, precision_score, roc_curve, average_precision_score,PrecisionRecallDisplay,precision_recall_curve

from sklearn.linear_model import LinearRegression, Ridge, Lasso, LogisticRegression, LogisticRegressionCV, Perceptron

from sklearn.svm import LinearSVC, SVR

from sklearn.ensemble import RandomForestRegressor

from sklearn.feature_selection import SelectFromModel, VarianceThreshold, RFE, SelectPercentile, SelectKBest

from sklearn.tree import DecisionTreeClassifier

In [ ]:
import os
print(os.listdir("../input"))


## - **Import Data Set**
## - **Combine train and test**
## - Observe the main data characteristics (size, types of variables, missing data, etc)

In [ ]:
## Importing Data Set
train = pd.read_csv('../input/titanic/train.csv')
test = pd.read_csv('../input/titanic/test.csv')

## Combining train and test data. Later they can be separated again for ML 
all_data =pd.concat([train,test],axis=0)


In [ ]:
all_data.head()

# Missing data

In [ ]:
pd.DataFrame(all_data.isnull().sum()).T

### Column 'Age'

In [ ]:
all_data[all_data['Age'].isnull()].head()

In [ ]:
all_data.groupby(['Sex','Pclass'])['Age'].mean()

In [ ]:
sns.barplot(x='Sex', y='Age', hue = 'Pclass',data=all_data);

In [ ]:
####################################################################################

def impute_age(cols):
    Age = cols[0]
    Pclass = cols[1]
    Sex = cols[2]
    
    if pd.isnull(Age):
        
        if (Pclass == 1) & (Sex == 'male'):
            return 41
        elif (Pclass == 2) & (Sex == 'male'):
            return 31
        elif (Pclass == 3) & (Sex == 'male'):
            return 26
        elif (Pclass == 1) & (Sex == 'female'):
            return 37
        elif (Pclass == 2) & (Sex == 'female'):
            return 27
        else:
            return 22
        
    else:
        return Age
    
    
all_data['Age'] = all_data[['Age','Pclass','Sex']].apply(impute_age, axis=1)


bins = [0, 16, 32, 48, 64, 80]

labels = [1,2,3,4,5]

all_data['Age_bin'] = pd.cut(all_data['Age'], bins = bins, labels=labels)

all_data['Baby_ind'] = np.where(all_data['Age'] < 6, 1, 0)

####################################################################################

all_data['Embarked'] = all_data['Embarked'].fillna('S')

####################################################################################

all_data['Fare'] = all_data['Fare'].fillna(13.0)

fare_bins = [0, 7.91, 14.45, 31,5000]

fare_labels = [0,1,2,3]

all_data['Fare_bin'] = pd.cut(all_data['Fare'], bins = fare_bins, labels=fare_labels)



####################################################################################

all_data['Cabin_ind'] = np.where(all_data['Cabin'].isnull(), 0, 1)

####################################################################################

all_data["Title"] = all_data["Name"].apply(lambda x: x.split(", ")[-1].split(".")[0])

all_data['Title'] = all_data['Title'].replace(['Lady', 'the Countess','Capt', 'Col','Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Officer')
# all_data['Title'] = all_data['Title'].replace(['Lady', 'the Countess', 'Don', 'Sir', 'Jonkheer', 'Dona'], 'Royal')
all_data['Title'] = all_data['Title'].replace(['Mlle', 'Ms'], 'Miss')
all_data['Title'] = all_data['Title'].replace(['Mme'], 'Mrs')

####################################################################################

#Family Size & Alone 
all_data['Family_Size'] = all_data['SibSp'] + all_data['Parch'] + 1
all_data['IsAlone'] = 0
all_data.loc[all_data['Family_Size']==1, 'IsAlone'] = 1

binss = [0, 2, 4, 20]

labelss = [1,2,3]

all_data['Family_Size_bin'] = pd.cut(all_data['Family_Size'], bins = binss, labels=labelss)

# all_data['Title'].value_counts()

all_data.head()

# Prepare for ML

In [ ]:
#pd.get_dummies(df, columns = ["Survived","Sex",'Embarked','Pclass'], prefix = ['sur_','sex_','em_','cls'])

In [ ]:
all_data_1 = all_data.drop(['PassengerId', 'Name', 'Cabin','Ticket','Fare','SibSp','Parch','Age','Family_Size'], axis = 1)
all_data_1 = all_data_1.astype({"Pclass": str})

all_dummies = pd.get_dummies(all_data_1, drop_first = True)


all_dummies.head()

In [ ]:
all_train = all_dummies[all_dummies['Survived'].notna()]

all_test = all_dummies[all_dummies['Survived'].isna()]


X_train, X_test, y_train, y_test = train_test_split(all_train.drop(['Survived'],axis=1), 
                                                    all_train['Survived'], test_size=0.30, 
                                                    random_state=101, stratify = all_train['Survived'])

In [ ]:
X_train.shape,y_train.shape 

# pipeline(Perceptron) with gridsearch

In [ ]:


pca = PCA()

nn=Perceptron()

pipe = Pipeline(steps=[("pca", pca), ("nn", nn)])

param_grid = {
    "pca__n_components": np.arange(1,21),
    "nn__penalty" : ['l2','l1','elasticnet'],
    "nn__alpha" :np.logspace(-4, 4, 7),
}


gridsearch_lr = GridSearchCV(estimator=pipe, 
                           param_grid=param_grid,
                           scoring='accuracy',
                           cv=5,
#                           verbose=2,
                           n_jobs=-1)

gridsearch_lr.fit(X_train,y_train)


best_estimator = gridsearch_lr.best_estimator_
best_accuracy = gridsearch_lr.best_score_
best_parameters = gridsearch_lr.best_params_

print('Best Estimator:', best_estimator)
print('Best Accuracy: {:.2f}%'.format(best_accuracy*100))
print('Best Parameters:', best_parameters)

#pd.DataFrame(griedsearch.cv_results_).head()



# Plot the PCA spectrum
pca.fit(X_train)

fig, (ax0, ax1) = plt.subplots(nrows=1,ncols=2, sharex=True, figsize=(12, 4))
ax0.plot(
    np.arange(1, pca.n_components_ + 1), pca.explained_variance_ratio_, "+", linewidth=2
)
ax0.set_ylabel("PCA explained variance ratio")

ax0.axvline(
    gridsearch_lr.best_estimator_.named_steps["pca"].n_components,
    linestyle=":",
    label="n_components chosen",
)
ax0.legend(prop=dict(size=12))

# For each number of components, find the best classifier results
results = pd.DataFrame(gridsearch_lr.cv_results_)
components_col = "param_pca__n_components"
best_clfs = results.groupby(components_col).apply(
    lambda g: g.nlargest(1, "mean_test_score")
)

best_clfs.plot(
    x=components_col, y="mean_test_score", yerr="std_test_score", legend=False, ax=ax1
)
ax1.set_ylabel("Classification accuracy (val)")
ax1.set_xlabel("n_components")

plt.xlim(-1, 22)

plt.tight_layout()
plt.show()

In [ ]:

pca = PCA(n_components=20)

nn=Perceptron(alpha=0.002154, penalty= 'elasticnet')

pipe = Pipeline(steps=[("pca", pca), ("logistic", logistic)])

pipe.fit(X_train, y_train)
print('Training score:', pipe.score(X_train,y_train))
p_predict= pipe.predict(X_test)

print(classification_report(y_test, p_predict))

print(confusion_matrix(y_test, p_predict))

# pipeline(Logistic Regression) with gridsearch

In [ ]:
pca = PCA()

log_reg = LogisticRegression(max_iter=1000)

pipe = Pipeline(steps=[("pca", pca), ("logistic", log_reg)])

param_grid = {
    "pca__n_components": np.arange(1,21),
    "logistic__C": np.logspace(-4, 4, 7),
    "logistic__fit_intercept" :[True,False],
}


gridsearch_lr = GridSearchCV(estimator=pipe, 
                           param_grid=param_grid,
                           scoring='accuracy',
                           cv=5,
#                           verbose=2,
                           n_jobs=-1)

gridsearch_lr.fit(X_train,y_train)


best_estimator = gridsearch_lr.best_estimator_
best_accuracy = gridsearch_lr.best_score_
best_parameters = gridsearch_lr.best_params_

print('Best Estimator:', best_estimator)
print('Best Accuracy: {:.2f}%'.format(best_accuracy*100))
print('Best Parameters:', best_parameters)

#pd.DataFrame(griedsearch.cv_results_).head()



# Plot the PCA spectrum
pca.fit(X_train)

fig, (ax0, ax1) = plt.subplots(nrows=1,ncols=2, sharex=True, figsize=(12, 4))
ax0.plot(
    np.arange(1, pca.n_components_ + 1), pca.explained_variance_ratio_, "+", linewidth=2
)
ax0.set_ylabel("PCA explained variance ratio")

ax0.axvline(
    gridsearch_lr.best_estimator_.named_steps["pca"].n_components,
    linestyle=":",
    label="n_components chosen",
)
ax0.legend(prop=dict(size=12))

# For each number of components, find the best classifier results
results = pd.DataFrame(gridsearch_lr.cv_results_)
components_col = "param_pca__n_components"
best_clfs = results.groupby(components_col).apply(
    lambda g: g.nlargest(1, "mean_test_score")
)

best_clfs.plot(
    x=components_col, y="mean_test_score", yerr="std_test_score", legend=False, ax=ax1
)
ax1.set_ylabel("Classification accuracy (val)")
ax1.set_xlabel("n_components")

plt.xlim(-1, 22)

plt.tight_layout()
plt.show()

In [ ]:
pca = PCA(n_components=16)

logistic = LogisticRegression(C = 1, max_iter=10000, fit_intercept = True, tol=0.0001)

pipe = Pipeline(steps=[("pca", pca), ("logistic", logistic)])

pipe.fit(X_train, y_train)
print('Training score:', pipe.score(X_train,y_train))
p_predict= pipe.predict(X_test)

print(classification_report(y_test, p_predict))

print(confusion_matrix(y_test, p_predict))

In [ ]:
log_reg = LogisticRegression(C = 0.1,fit_intercept = False)

log_reg.fit(X_train,y_train)

print('Training score:', log_reg.score(X_train,y_train))

fig,(ax1,ax2)=plt.subplots(1,2, figsize=(10,5))
PrecisionRecallDisplay.from_estimator(log_reg, X_train, y_train, ax = ax1)



pred_lr = log_reg.predict(X_test)


print(classification_report(y_test, pred_lr))

print(confusion_matrix(y_test, pred_lr))

print(precision_score(y_test, pred_lr))

print(recall_score(y_test, pred_lr))
print(roc_curve(y_test, pred_lr))
print(roc_auc_score(y_test, pred_lr))
print(average_precision_score(y_test, pred_lr))

y_pred = log_reg.predict_proba(X_test)[:, 1]
PrecisionRecallDisplay.from_predictions(y_test, y_pred, ax=ax2)

# Pipeline(Random Forest Classifier) with gridsearch

In [ ]:
pca.get_params().keys()
pipe.get_params().keys()

In [ ]:
pca = PCA()
rfc = RandomForestClassifier()

pipe = Pipeline(steps=[("pca", pca), ("rfc", rfc)])

param_grid = {
    "pca__n_components": [2,5,8,10,11,14,15,16,20],
    'rfc__n_estimators': [50, 100, 250, 300],
    'rfc__max_depth': [None, 100, 200, 300, 400],
    'rfc__criterion' :['gini', 'entropy']
}


gridsearch_rfc = GridSearchCV(estimator=pipe, 
                           param_grid=param_grid,
                           scoring='accuracy',
                           cv=3,
#                           verbose=2,
                           n_jobs=-1)

gridsearch_rfc.fit(X_train,y_train)


best_estimator = gridsearch_rfc.best_estimator_
best_accuracy = gridsearch_rfc.best_score_
best_parameters = gridsearch_rfc.best_params_

print('Best Estimator:', best_estimator)
print('Best Accuracy: {:.2f}%'.format(best_accuracy*100))
print('Best Parameters:', best_parameters)

#pd.DataFrame(griedsearch.cv_results_).head()



# Plot the PCA spectrum
pca.fit(X_train)

fig, (ax0, ax1) = plt.subplots(nrows=1,ncols=2, sharex=True, figsize=(12, 4))
ax0.plot(
    np.arange(1, pca.n_components_ + 1), pca.explained_variance_ratio_, "+", linewidth=2
)
ax0.set_ylabel("PCA explained variance ratio")

ax0.axvline(
    gridsearch_rfc.best_estimator_.named_steps["pca"].n_components,
    linestyle=":",
    label="n_components chosen",
)
ax0.legend(prop=dict(size=12))

# For each number of components, find the best classifier results
results = pd.DataFrame(gridsearch_rfc.cv_results_)
components_col = "param_pca__n_components"
best_clfs = results.groupby(components_col).apply(
    lambda g: g.nlargest(1, "mean_test_score")
)

best_clfs.plot(
    x=components_col, y="mean_test_score", yerr="std_test_score", legend=False, ax=ax1
)
ax1.set_ylabel("Classification accuracy (val)")
ax1.set_xlabel("n_components")

plt.xlim(-1, 22)

plt.tight_layout()
plt.show()

In [ ]:
pca = PCA(n_components=20)

rfc = RandomForestClassifier(criterion= 'entropy', max_depth= 300, n_estimators=250)

pipe = Pipeline(steps=[("pca", pca), ("rfc", rfc)])

pipe.fit(X_train, y_train)
print('Training score:', pipe.score(X_train,y_train))
rfc_predict= pipe.predict(X_test)

print(classification_report(y_test, rfc_predict))

print(confusion_matrix(y_test, rfc_predict))

# K-Fold Cross Validation with Random Forest:

In [ ]:
rfc = RandomForestClassifier(criterion= 'gini', max_depth= 400, n_estimators=250)
scores_rfc = cross_val_score(rfc, X_train, y_train, cv=10, scoring = "accuracy")
print("Scores:", scores_rfc)
print("Mean:", scores_rfc.mean())
print("Standard Deviation:", scores_rfc.std())

In [ ]:
KNeighborsClassifier().get_params().keys()

# Pipeline(Random Forest) with gridsearch

In [ ]:
pca = PCA()
knn = KNeighborsClassifier()


pipe = Pipeline(steps=[("pca", pca), ("knn", knn)])

param_grid = {
    "pca__n_components": np.arange(1,21),
    'knn__n_neighbors': np.arange(1,40, 5),
#     'rfc__max_depth': [None, 100, 200, 300, 400],
#     'rfc__criterion' :['gini', 'entropy']
}


gridsearch_knn = GridSearchCV(estimator=pipe, 
                           param_grid=param_grid,
                           scoring='accuracy',
                           cv=3,
#                           verbose=2,
                           n_jobs=-1)

gridsearch_knn.fit(X_train,y_train)


best_estimator = gridsearch_knn.best_estimator_
best_accuracy = gridsearch_knn.best_score_
best_parameters = gridsearch_knn.best_params_

print('Best Estimator:', best_estimator)
print('Best Accuracy: {:.2f}%'.format(best_accuracy*100))
print('Best Parameters:', best_parameters)

#pd.DataFrame(griedsearch.cv_results_).head()



# Plot the PCA spectrum
pca.fit(X_train)

fig, (ax0, ax1) = plt.subplots(nrows=1,ncols=2, sharex=True, figsize=(12, 4))
ax0.plot(
    np.arange(1, pca.n_components_ + 1), pca.explained_variance_ratio_, "+", linewidth=2
)
ax0.set_ylabel("PCA explained variance ratio")

ax0.axvline(
    gridsearch_knn.best_estimator_.named_steps["pca"].n_components,
    linestyle=":",
    label="n_components chosen",
)
ax0.legend(prop=dict(size=12))

# For each number of components, find the best classifier results
results = pd.DataFrame(gridsearch_knn.cv_results_)
components_col = "param_pca__n_components"
best_clfs = results.groupby(components_col).apply(
    lambda g: g.nlargest(1, "mean_test_score")
)

best_clfs.plot(
    x=components_col, y="mean_test_score", yerr="std_test_score", legend=False, ax=ax1
)
ax1.set_ylabel("Classification accuracy (val)")
ax1.set_xlabel("n_components")

plt.xlim(-1, 22)

plt.tight_layout()
plt.show()

In [ ]:
pca = PCA(n_components=16)

knn = KNeighborsClassifier(n_neighbors=11)

pipe = Pipeline(steps=[("pca", pca), ("knn", knn)])

pipe.fit(X_train, y_train)
print('Training score:', pipe.score(X_train,y_train))
print(round(pipe.score(X_train, y_train) * 100, 2))

knn_predict= pipe.predict(X_test)

print(classification_report(y_test, knn_predict))

print(confusion_matrix(y_test, knn_predict))



## Elbow Method

In [ ]:
error_rate = []

for i in range(1,40):
    knn = KNeighborsClassifier(n_neighbors=i)
    knn.fit(X_train,y_train)
    pred_i = knn.predict(X_test)
    error_rate.append((knn.score(X_train,y_train)))
                     
# for i in range(1,40):
#     print("k: ", format(i))
#     print("error_rate", error_rate[i-1])

import matplotlib.pyplot as plt
from matplotlib import style

plt.figure(figsize=(16,8))
plt.plot(range(1,40),error_rate,color='blue', linestyle='dashed', marker='o',
          markerfacecolor='gray', markersize=10)
plt.title('Score vs. K Value')
plt.xlabel('K')
plt.ylabel('Score')

# Submission

In [ ]:
pca = PCA(n_components=16)

logistic = LogisticRegression(C = 1, max_iter=10000, fit_intercept = True, tol=0.0001)

pipe = Pipeline(steps=[("pca", pca), ("logistic", logistic)])

pipe.fit(X_train, y_train)
print('Training score:', pipe.score(X_train,y_train))

p_predict= pipe.predict(all_test.drop('Survived', axis=1))


# print(classification_report(y_test, knn_predict))

# print(confusion_matrix(y_test, knn_predict))

In [ ]:
submission_df = pd.DataFrame({
    'PassengerId' : test['PassengerId'], 
    'Survived' : p_predict
})

submission_df['Survived'] = submission_df['Survived'].astype('int')


submission_df.to_csv('./submission.csv', index=False)

submission_df.head()